## Importing libraries

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import Model, Input, regularizers
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, UpSampling2D, Reshape
from tensorflow.keras.callbacks import EarlyStopping
from keras.preprocessing import image
from tensorflow.keras.utils import load_img,img_to_array
import glob
from tqdm import tqdm
import warnings;
warnings.filterwarnings('ignore')
import pickle
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Flatten,Reshape
from tensorflow.keras.optimizers import SGD

## Loading blei

In [ ]:

file = open('blemished_train.pkl','rb')

X_train = pickle.load(file)
file.close()

In [ ]:
file = open('imagenet_subset.pkl','rb')

y_train = pickle.load(file)
# y_train = y_train[3000:6000]

file.close()

In [ ]:
file = open('blemished_test.pkl','rb')

X_test = pickle.load(file)

file.close()

In [ ]:
file = open('imagenet_testset.pkl','rb')

y_test = pickle.load(file)
# y_test = y_test[300:600]

file.close()

In [ ]:
file = open('blemished_val.pkl','rb')

X_val = pickle.load(file)
# y_test = y_test[300:600]

file.close()

In [ ]:
file = open('imagenet_valset.pkl','rb')

y_val = pickle.load(file)
# y_test = y_test[300:600]

file.close()

In [ ]:
import cv2
import pandas as pd

In [ ]:
def resize_images(img, new_width, new_height):
    size = (new_width, new_height)
    resized_img = cv2.resize(img, size)
    return resized_img

In [ ]:
resized_img_Xtrain = [resize_images(img, new_width=80, new_height=80) for img in X_train]
resized_img_Ytrain = [resize_images(img, new_width=80, new_height=80) for img in y_train]
resized_img_Xtest = [resize_images(img, new_width=80, new_height=80) for img in X_test]
resized_img_Ytest = [resize_images(img, new_width=80, new_height=80) for img in y_test]
resized_img_Xval = [resize_images(img, new_width=80, new_height=80) for img in X_val]
resized_img_Yval = [resize_images(img, new_width=80, new_height=80) for img in y_val]

In [ ]:
plt.imshow(resized_img_Xval[0])

In [ ]:
# for i in range(len(resized_img_Yval)):
#   if resized_img_Yval[i].shape[-1] != 3:
#     pass

#   print(resized_img_Yval[i].shape)

In [ ]:
# get low resolution images for the training set
# train_x_px = []
train_x = np.array(resized_img_Ytrain)
train_x = train_x/255.0
test_x = np.array(resized_img_Ytest)
test_x = test_x/255.0
val_x = np.array(resized_img_Yval)
val_x = val_x/255.0
# for i in range(np.array(train_x).shape[0]):
#   temp = pixalate_image(train_x[i,:,:,:])
#   train_x_px.append(temp)

train_x_px = np.array(resized_img_Xtrain)
train_x_px = train_x_px/255.0


# get low resolution images for the validation set
# val_x_px = []

# for i in range(val_x.shape[0]):
#   temp = pixalate_image(val_x[i,:,:,:])
#   val_x_px.append(temp)

test_x_px = np.array(resized_img_Xtest)
test_x_px = test_x_px/255.0

val_x_px = np.array(resized_img_Xval)
val_x_px = val_x_px/255.0

In [ ]:
encoder = Sequential()

encoder.add(Flatten(input_shape=[80,80,3]))
encoder.add(Dense(9600,activation="relu"))
encoder.add(Dense(4800,activation="relu"))
encoder.add(Dense(2400,activation="relu"))
encoder.add(Dense(1200,activation="relu"))
encoder.add(Dense(600,activation="relu"))


decoder = Sequential()

decoder.add(Dense(1200,input_shape=[600],activation='relu'))
decoder.add(Dense(2400,activation='relu'))
decoder.add(Dense(4800,activation='relu'))
decoder.add(Dense(9600,activation='relu'))
decoder.add(Dense(80*80*3,activation='sigmoid',input_shape = [9600]))
decoder.add(Reshape([80,80,3]))

simple_autoencoder = Sequential([encoder,decoder])

autoencoder.compile(optimizer='adam', loss='mse')


In [ ]:
early_stopper = EarlyStopping(monitor='val_loss', min_delta=0.0001, patience=4, verbose=1, mode='auto')

simple_autoencoder.fit(train_x_px, train_x,
            epochs=50,
            batch_size=32,
            shuffle=True,
            validation_data=(val_x_px, val_x),
            callbacks=[early_stopper]
            )

In [ ]:
Input_img = Input(shape=(80, 80, 3))  
# Reshape_img = Reshape((80, 80), input_shape=(128,128))(Input_img)    
#encoding architecture
x1 = Conv2D(256, (3, 3), activation='relu', padding='same')(Input_img)
x2 = Conv2D(128, (3, 3), activation='relu', padding='same')(x1)
x2 = MaxPool2D( (2, 2))(x2)
encoded = Conv2D(64, (3, 3), activation='relu', padding='same')(x2)

# decoding architecture
x3 = Conv2D(64, (3, 3), activation='relu', padding='same')(encoded)
x3 = UpSampling2D((2, 2))(x3)
x2 = Conv2D(128, (3, 3), activation='relu', padding='same')(x3)
x1 = Conv2D(256, (3, 3), activation='relu', padding='same')(x2)
decoded = Conv2D(3, (3, 3), padding='same')(x1)

autoencoder = Model(Input_img, decoded)
sgd = tf.keras.optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.5, nesterov=True)
autoencoder.compile(optimizer=sgd, loss='mse')

In [ ]:
early_stopper = EarlyStopping(monitor='val_loss', min_delta=0.0001, patience=4, verbose=1, mode='auto')

autoencoder.fit(train_x_px, train_x,
            epochs=50,
            batch_size=32,
            shuffle=True,
            validation_data=(val_x_px, val_x),
            callbacks=[early_stopper]
            )

In [ ]:
predictions = autoencoder.predict(test_x_px)

In [ ]:
predictions.shape

In [ ]:
n = 5
plt.figure(figsize= (20,10))

for i in range(n):
  ax = plt.subplot(2, n, i+1)
  plt.imshow(test_x_px[i+10])
  ax.get_xaxis().set_visible(False)
  ax.get_yaxis().set_visible(False)

  ax = plt.subplot(2, n, i+1+n)
  plt.imshow(predictions[i+10])
  ax.get_xaxis().set_visible(False)
  ax.get_yaxis().set_visible(False)

plt.show()

In [ ]:
# encoder = tf.keras.models.Sequential()

# encoder.add(tf.keras.layers.Conv2D(32, 3, strides=1, padding='same', activation='relu', input_shape=(128,128,3)))
# encoder.add(tf.keras.layers.MaxPooling2D(2, strides=2))

# encoder.add(tf.keras.layers.Conv2D(64, 3, strides=1, padding='same', activation='relu'))
# encoder.add(tf.keras.layers.MaxPooling2D(2, strides=2))

# encoder.add(tf.keras.layers.Conv2D(128, 3, strides=1, padding='same', activation='relu'))
# encoder.add(tf.keras.layers.MaxPooling2D(2, strides=2))

# encoder.summary()


In [ ]:
# decoder = tf.keras.models.models.Sequential()

# decoder.add(tf.keras.models.layers.Conv2DTranspose(128, 3, strides=1, padding='same', activation='relu', input_shape= (16,16,128)))
# decoder.add(tf.keras.models.layers.2D(2))

# decoder.add(tf.keras.models.layers.Conv2D(16, 3, strides=1, padding='same', activation='relu'))
# decoder.add(tf.keras.models.layers.UpSampling2D(2))

# decoder.add(tf.keras.models.layers.Conv2D(3, 3, strides=1, padding='same', activation='relu'))
# decoder.add(tf.keras.models.layers.UpSampling2D(2))

# decoder.summary()

In [ ]:
# import numpy as np
# from keras.layers import UpSampling2D,Conv2DTranspose
# from keras.models import Sequential
# model = Sequential()
# model.add(Conv2DTranspose(2, 3, input_shape=(3,3,1)))

# x = np.arange(9).reshape(1,3,3,1)
# x[0,:,:,0]  # this is what x looks like initially

# y = model.predict(x)
# y[0,:,:,0] # this is what it looks like after upsampling

In [ ]:
# import numpy as np
# from keras.layers import UpSampling2D,Conv2DTranspose
# from keras.models import Sequential
# model = Sequential()
# # model.add(Conv2DTranspose(2, 3, input_shape=(3,3,1)))
# model.add(UpSampling2D(size=(2,2), input_shape=(3,3,1)))

# x = np.arange(9).reshape(1,3,3,1)
# x[0,:,:,0]  # this is what x looks like initially

# y = model.predict(x)
# y[0,:,:,0] # this is what it looks like after upsampling

In [ ]:
df = pd.DataFrame(a_e.history)

In [ ]:
df.plot()